# Загружаем необходимые библиотеки.

In [56]:
import json, pandas as pd, re, nltk

from collections import Counter

from nltk.tokenize import word_tokenize
nltk.download('punkt')

from nltk.corpus import stopwords
nltk.download("stopwords")
stopwords_russian = stopwords.words("russian")

from tqdm import tqdm
tqdm.pandas()

[nltk_data] Downloading package punkt to
[nltk_data]     /home/boyarskikhae/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/boyarskikhae/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Получаем данные.

In [57]:
df_train = pd.read_json("../train.json")
df_test = pd.read_json("../test.json")

# Explanatory data analysis.

## df_train.

In [58]:
df_train.info()

print("\n")
print(df_train["label"].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1799 non-null   int64 
 1   text            1799 non-null   object
 2   label           1799 non-null   object
 3   extracted_part  1799 non-null   object
dtypes: int64(1), object(3)
memory usage: 56.3+ KB


обеспечение исполнения контракта        988
обеспечение гарантийных обязательств    811
Name: label, dtype: int64


## Проверим, какие слова чаще всего встречаются в колонке "extracted_part".

In [59]:
extracted_parts = list(df_train["extracted_part"])
extracted_text = []

for part in extracted_parts:
    extracted_text.append(part["text"][0])

def process_extracted_text(text):
    text = " ".join(text)
    text = re.sub(r"[^А-Яа-я]", " ", text)
    word_tokens = word_tokenize(text)
    return ([word_token for word_token in word_tokens if word_token not in stopwords_russian]) 

extracted_text = process_extracted_text(extracted_text)

Counter(extracted_text).most_common(20)

[('исполнения', 1046),
 ('обеспечения', 1003),
 ('Размер', 930),
 ('цены', 924),
 ('контракта', 720),
 ('договора', 696),
 ('начальной', 680),
 ('максимальной', 679),
 ('обязательств', 542),
 ('гарантийных', 525),
 ('размере', 453),
 ('составляет', 431),
 ('Контракта', 338),
 ('Обеспечение', 271),
 ('Российский', 258),
 ('рубль', 258),
 ('обеспечение', 249),
 ('устанавливается', 197),
 ('Договора', 181),
 ('рублей', 178)]

Дополнительно пробив такие популярные слова как "Размер", можно увидеть некоторые закономерности, например, в вычленненом тексте всегда присутствует какое-либо число.

## df_test.

In [60]:
df_test.info()

print("\n")
print(df_test["label"].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      318 non-null    int64 
 1   text    318 non-null    object
 2   label   318 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.6+ KB


обеспечение исполнения контракта        175
обеспечение гарантийных обязательств    143
Name: label, dtype: int64


# Работаем с регулярными выражениями.

## Я заметил два паттерна, которые преобладают в наших данных. Первый паттерн начинается со слова "Размер", а второй - со слова "Обеспечение".

In [61]:
oik_razmer = re.compile("[Рр]азмер обеспечени[а-я] исполнени[а-я] ([Кк]онтракт[а-я]|[Дд]оговор[а-я])[^\d]+(\d|\.|\,)+( Российский)?[^А-Я.]+[^А-Я. \d]+( Договора| Контракта)?\.?")
ogb_razmer = re.compile("(Поставщик|[Рр]азмер) обеспечени[а-я] гарантийных обязательств[^\d]+(\d|\.|\,)+[^А-Я.]+(Российский рубль)?(НМЦК)?(Договора|Контракта)?\.?")
oik_caps = re.compile("[Оо]беспечени[а-я] исполнени[а-я] (настоящего)?\s?([Кк]онтракт[а-я]|[Дд]оговор[а-я])[^\d]+(\d|\.|\,)+[^.,:;]+\.?")
ogb_caps = re.compile("[Оо]беспечени[а-я] гарантийных обязательств[^\d]+(\.|\,|\d)+[^.,:;]+\.?")

 ## Используем костыль, чтобы получить паттерн, оканчивающийся на слово "контракт", потому что я не смог вычленить его с помощью регулярных выражений.

In [62]:
def sub_text(text):
    sub_text = re.sub("контракта", "Контракта", text)
    extracted_part = oik_razmer.search(sub_text).group()
    extracted_part = re.sub("Контракта", "контракта", extracted_part)
    return [extracted_part]

## Код функции, которая будет формировать ответ именно в том виде, который прописан в техническом задании.

In [63]:
def cut_the_text(text, label):
    if label == "обеспечение исполнения контракта":
        try:
            extracted_part = oik_razmer.search(text).group()
            answer_end = [list(oik_razmer.search(text).span())[1]]
            string_length = len(extracted_part)
            # Начало работы костыля.
            if re.search(r".* начальной \(максимальной\) цены контракта.*", extracted_part) != None:
                extracted_part = sub_text(text)
                new_string_length = len(extracted_part[0])
                answer_end = [answer_end[0] - (string_length - new_string_length)]
            else: extracted_part = [oik_razmer.search(text).group()]
            # Конец работы костыля.
            answer_start = [list(oik_razmer.search(text).span())[0]]
        except AttributeError:
            try:
                extracted_part = [oik_caps.search(text).group()]
                answer_start = [list(oik_caps.search(text).span())[0]]
                answer_end = [list(oik_caps.search(text).span())[1]]
            except AttributeError:
                extracted_part = [""]
                answer_start = [0]
                answer_end = [0]
    else:
        try:
            extracted_part = [ogb_razmer.search(text).group()]
            answer_start = [list(ogb_razmer.search(text).span())[0]]
            answer_end = [list(ogb_razmer.search(text).span())[1]]
        except AttributeError:
            try:
                extracted_part = [ogb_caps.search(text).group()]
                answer_start = [list(ogb_caps.search(text).span())[0]]
                answer_end = [list(ogb_caps.search(text).span())[1]]
            except AttributeError:
                extracted_part = [""]
                answer_start = [0]
                answer_end = [0]
    return({"text": extracted_part, "answer_start": answer_start, "answer_end": answer_end})

## Применим функцию и посчитаем точность на df_train.

In [64]:
df_train["new_extracted_part"] = df_train.progress_apply(lambda x: cut_the_text(x["text"], x["label"]), axis = 1)

print("The score is ", len(df_train[df_train["extracted_part"] == df_train["new_extracted_part"]])/len(df_train)*100)

100%|████████████████████████████████████| 1799/1799 [00:00<00:00, 10940.64it/s]

The score is  54.14118954974986


## Применим функцию к df_test.

In [65]:
df_test["extracted_part"] = df_test.progress_apply(lambda x: cut_the_text(x["text"], x["label"]), axis = 1)

100%|██████████████████████████████████████| 318/318 [00:00<00:00, 11873.21it/s]


# Формируем ответ.

In [66]:
df_test.to_json("../predictions.json")